In [1]:
import torch
from torch import nn




class SPPF(nn.Module):
    # Spatial Pyramid Pooling - Fast (SPPF) layer for YOLOv5 by Glenn Jocher
    def __init__(self, c1, c2, k=5):  # equivalent to SPP(k=(5, 9, 13))
        super().__init__()
        c_ = c1 // 2  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c_ * 4, c2, 1, 1)
        self.m = nn.MaxPool2d(kernel_size=k, stride=1, padding=k // 2)
        # #修改成3*3核
        # self.mk = nn.MaxPool2d(kernel_size=3, stride=1, padding=3 // 2)

    def forward(self, x):
        x = self.cv1(x)
        # with warnings.catch_warnings():
        #     warnings.simplefilter('ignore')  # suppress torch 1.9.0 max_pool2d() warning
            # y1 = self.m(x)
            # y2 = self.m(y1)
            # return self.cv2(torch.cat((x, y1, y2, self.m(y2)), 1))
        #修改成3*3核
        y1 = self.m(x)
        y2 = self.m(y1)
        return self.cv2(torch.cat((x, y1, y2, self.m(y2)), 1))


class Conv(nn.Module):
    # Standard convolution with args(ch_in, ch_out, kernel, stride, padding, groups, dilation, activation)
    default_act = nn.SiLU()  # default activation

    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, d=1, act=True):
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p, d), groups=g, dilation=d, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = self.default_act if act is True else act if isinstance(act, nn.Module) else nn.Identity()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def forward_fuse(self, x):
        return self.act(self.conv(x))
    
    
def autopad(k, p=None, d=1):  # kernel, padding, dilation
    # Pad to 'same' shape outputs
    if d > 1:
        k = d * (k - 1) + 1 if isinstance(k, int) else [d * (x - 1) + 1 for x in k]  # actual kernel-size
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p

In [ ]:
# 创建 SPP 模块，设置金字塔层级
spp = SPPF(levels=[1, 2, 4])

# 输入图像1，尺寸为 640x640
image1 = torch.randn(1, 3, 640, 640)
spp_features1 = spp(image1)

# 输入图像2，尺寸为 320x320
image2 = torch.randn(1, 3, 320, 320)
spp_features2 = spp(image2)

# 将 SPP 特征向量作为模型的输入进行训练或预测
model_input = torch.cat([spp_features1, spp_features2], dim=1)